# Final Project 
## Raw Analysis

In [3]:
# Import all the needed modules for this exercise
import pandas as pd
import country_converter as coco
import requests # For downloading the website
from bs4 import BeautifulSoup # For parsing the website
import warnings # import warnings 
warnings.filterwarnings("ignore") # ignore global warnings
import time # To put the system to sleep
import random # for random numbers
import sqlite3
import matplotlib.pyplot as pt
from plotnine import *
import numpy as np
import openpyxl


In [43]:
# Load first dataset (Source: https://www.kff.org/statedata/custom-state-report/?32139-year-=2019&view=3&i=32148~460185~32141~32139~32132~32136&g=us~al~ak~az~ar~ca~co~ct~de~dc~fl~ga~hi~id~il~in~ia~ks~ky~la~me~md~ma~mi~mn~ms~mo~mt~ne~nv~nh~nj~nm~ny~nc~nd~oh~ok~or~pa~ri~sc~sd~tn~tx~ut~vt~va~wa~wv~wi~wy)
Poverty_Expenditure = pd.read_excel('/Users/abhatti/Documents/masterInScience/PPOL564/Final Project/Final-Project-Unemployment-Analysis/data/statehealthfacts.xlsx')
dta = Poverty_Expenditure.melt(id_vars="Unnamed: 0")
dta = dta.rename(columns={"Unnamed: 0": "Features", "variable": "State"})
pv = dta.pivot_table(index=["State"], 
                    columns='Features', 
                    values='value')
Poverty_Expenditure = pv

In [54]:
Poverty_Expenditure.head()

Features,Poverty Rate by Age: 65+,Poverty Rate by Age: Adults 19-64,Poverty Rate by Age: Children 0-18,Poverty Rate by Age: Total,Poverty Rate by Race/Ethnicity: American Indian/Alaska Native,Poverty Rate by Race/Ethnicity: Asian/Native Hawaiian and Pacific Islander,Poverty Rate by Race/Ethnicity: Black,Poverty Rate by Race/Ethnicity: Hispanic,Poverty Rate by Race/Ethnicity: Multiple Races,Poverty Rate by Race/Ethnicity: Total,Poverty Rate by Race/Ethnicity: White,Total State Expenditures (in millions)
State,,,,,,,,,,,,
Alabama,0.105,0.148,0.208,0.154,0.156,0.107,0.236,0.244,0.186,0.154,0.115,28834.0
Alaska,0.083,0.101,0.133,0.107,0.240,NaN,NaN,0.149,0.069,0.107,0.069,11148.0
Arizona,0.091,0.126,0.187,0.134,0.290,0.115,0.175,0.182,0.153,0.134,0.092,38691.0
Arkansas,0.103,0.156,0.230,0.165,0.155,0.125,0.289,0.201,0.227,0.165,0.134,25639.0
California,0.104,0.106,0.157,0.118,0.203,0.090,0.193,0.150,0.107,0.118,0.084,300445.0


In [45]:
# Load unemployment data (12-Month Data (4/1/2020 – 03/31/2021)
# Improper payment data for the Unemployment Insurance (UI) program derived from the Benefit Accuracy Measurement (BAM) program for the 12-month period ending March 31, 2021. BAM is a quality control statistical survey used to identify errors and support corrective action in the state unemployment insurance (UI) system. Data includes improper payment percentages, total dollar amounts, and root cause by state.
Unemployment_Data = pd.read_excel('/Users/abhatti/Documents/masterInScience/PPOL564/Final Project/Final-Project-Unemployment-Analysis/data/2021 - Twelve Months Ending March 31.xlsx', sheet_name='Improper Payment Rates', header = 6, skipfooter = 6)
Unemployment_Data = Unemployment_Data.rename(columns={"ST": "StateCode", 
                                                      "Amount Paid": "Total Unemployment Paid", 
                                                      "(a)": "Overpayment Rate", 
                                                      "(c)": "Amount Overpaid", 
                                                      "(b)": "Underpayment Rate", 
                                                      "(d)": "Amount Underpaid", 
                                                      "(a)+(b)": "Improper Payment Rate", 
                                                      "(c)+(d)": "Amount Improperly Paid", 
                                                      "Work. Search.": "Overpayment Rate excl Work Search",
                                                      "Work. Search..1": "Amount Overpaid excl Work Search",
                                                      "Rate": "Fraud Rate"
                                                     })
Unemployment_Data.head()

,StateCode,Total Unemployment Paid,Overpayment Rate,Amount Overpaid,Underpayment Rate,Amount Underpaid,Improper Payment Rate,Amount Improperly Paid,Overpayment Rate excl Work Search,Amount Overpaid excl Work Search,Fraud Rate
0,AK,314026879,0.08134,25542322,0.04325,13582376,0.12459,39124699,0.06550,20569016,0.01785
1,AL,533312996,0.23671,126239472,0.00285,1519078,0.23956,127758550,0.23529,125482820,0.13475
2,AR,171995172,0.22442,38598834,0.01726,2968952,0.24168,41567786,0.22442,38598834,0.10602
3,AZ,919184394,0.25343,232951241,0.00000,0,0.25343,232951241,0.25343,232951241,0.14078
4,CA,16583022143,0.07375,1222937384,0.01559,258463590,0.08933,1481400974,0.07375,1222937384,0.03080


In [34]:
state_abbrev = pd.read_csv('/Users/abhatti/Documents/masterInScience/PPOL564/Final Project/Final-Project-Unemployment-Analysis/data/stateAbbrev.csv')

In [46]:
state_abbrev.head(2)

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK


In [49]:
Unemployment_Data = Unemployment_Data.merge(state_abbrev, left_on='StateCode', right_on='Code', how='left')

In [52]:
Unemployment_Data.head(2)

,StateCode,Total Unemployment Paid,Overpayment Rate,Amount Overpaid,Underpayment Rate,Amount Underpaid,Improper Payment Rate,Amount Improperly Paid,Overpayment Rate excl Work Search,Amount Overpaid excl Work Search,Fraud Rate,State,Abbrev,Code
0,AK,314026879,0.08134,25542322,0.04325,13582376,0.12459,39124699,0.06550,20569016,0.01785,Alaska,Alaska,AK
1,AL,533312996,0.23671,126239472,0.00285,1519078,0.23956,127758550,0.23529,125482820,0.13475,Alabama,Ala.,AL


In [53]:
Unemployment_Data.drop(['StateCode', 'Abbrev'], axis=1)

,Total Unemployment Paid,Overpayment Rate,Amount Overpaid,Underpayment Rate,Amount Underpaid,Improper Payment Rate,Amount Improperly Paid,Overpayment Rate excl Work Search,Amount Overpaid excl Work Search,Fraud Rate,State,Code
0,314026879,0.08134,25542322,0.04325,13582376,0.12459,39124699,0.06550,20569016,0.01785,Alaska,AK
1,533312996,0.23671,126239472,0.00285,1519078,0.23956,127758550,0.23529,125482820,0.13475,Alabama,AL
2,171995172,0.22442,38598834,0.01726,2968952,0.24168,41567786,0.22442,38598834,0.10602,Arkansas,AR
3,919184394,0.25343,232951241,0.00000,0,0.25343,232951241,0.25343,232951241,0.14078,Arizona,AZ
4,16583022143,0.07375,1222937384,0.01559,258463590,0.08933,1481400974,0.07375,1222937384,0.03080,California,CA
5,1810365726,0.34044,616325307,0.00919,16645676,0.34964,632970983,0.33092,599077809,0.06695,Colorado,CO
6,1901024320,0.17160,326213311,0.00125,2372476,0.17285,328585787,0.17160,326213311,0.04797,Connecticut,CT
7,440478492,0.07888,34745751,0.00621,2737474,0.08510,37483225,0.07888,34745751,0.02150,District of Columbia,DC
8,171023748,0.14167,24228688,0.00001,1417,0.14168,24230105,0.14167,24228688,0.02403,Delaware,DE
9,1985206685,0.39698,788083529,0.00234,4641656,0.39932,792725186,0.39698,788083529,0.06212,Florida,FL


In [55]:
fullData = Poverty_Expenditure.merge(Unemployment_Data, left_on='State', right_on='State', how='left')

In [56]:
fullData

,State,Poverty Rate by Age: 65+,Poverty Rate by Age: Adults 19-64,Poverty Rate by Age: Children 0-18,Poverty Rate by Age: Total,Poverty Rate by Race/Ethnicity: American Indian/Alaska Native,Poverty Rate by Race/Ethnicity: Asian/Native Hawaiian and Pacific Islander,Poverty Rate by Race/Ethnicity: Black,Poverty Rate by Race/Ethnicity: Hispanic,Poverty Rate by Race/Ethnicity: Multiple Races,...,Amount Overpaid,Underpayment Rate,Amount Underpaid,Improper Payment Rate,Amount Improperly Paid,Overpayment Rate excl Work Search,Amount Overpaid excl Work Search,Fraud Rate,Abbrev,Code
0,Alabama,0.105,0.148,0.208,0.154,0.156,0.107,0.236,0.244,0.186,...,1.262395e+08,0.00285,1519078.0,0.23956,1.277586e+08,0.23529,1.254828e+08,0.13475,Ala.,AL
1,Alaska,0.083,0.101,0.133,0.107,0.240,NaN,NaN,0.149,0.069,...,2.554232e+07,0.04325,13582376.0,0.12459,3.912470e+07,0.06550,2.056902e+07,0.01785,Alaska,AK
2,Arizona,0.091,0.126,0.187,0.134,0.290,0.115,0.175,0.182,0.153,...,2.329512e+08,0.00000,0.0,0.25343,2.329512e+08,0.25343,2.329512e+08,0.14078,Ariz.,AZ
3,Arkansas,0.103,0.156,0.230,0.165,0.155,0.125,0.289,0.201,0.227,...,3.859883e+07,0.01726,2968952.0,0.24168,4.156779e+07,0.22442,3.859883e+07,0.10602,Ark.,AR
4,California,0.104,0.106,0.157,0.118,0.203,0.090,0.193,0.150,0.107,...,1.222937e+09,0.01559,258463590.0,0.08933,1.481401e+09,0.07375,1.222937e+09,0.03080,Calif.,CA
5,Colorado,0.073,0.094,0.107,0.094,0.181,0.082,0.197,0.143,0.143,...,6.163253e+08,0.00919,16645676.0,0.34964,6.329710e+08,0.33092,5.990778e+08,0.06695,Colo.,CO
6,Connecticut,0.075,0.095,0.142,0.102,NaN,0.088,0.184,0.220,0.138,...,3.262133e+08,0.00125,2372476.0,0.17285,3.285858e+08,0.17160,3.262133e+08,0.04797,Conn.,CT
7,Delaware,0.075,0.103,0.155,0.109,NaN,0.084,0.173,0.211,NaN,...,2.422869e+07,0.00001,1417.0,0.14168,2.423010e+07,0.14167,2.422869e+07,0.02403,Del.,DE
8,Dist. of Columbia,0.144,0.124,0.207,0.143,NaN,0.126,0.234,0.089,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,0.107,0.116,0.178,0.127,0.139,0.102,0.198,0.157,0.147,...,7.880835e+08,0.00234,4641656.0,0.39932,7.927252e+08,0.39698,7.880835e+08,0.06212,Fla.,FL


In [33]:
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Firefox(executable_path=r'your\path\geckodriver.exe')
driver.get('http://eve-central.com/home/quicklook.html?typeid=34')

html = driver.page_source
soup = BeautifulSoup(html)

# check out the docs for the kinds of things you can do with 'find_all'
# this (untested) snippet should find tags with a specific class ID
# see: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
for tag in soup.find_all("a", class_="my_class"):
    print(tag.text)

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


In [25]:
# define the url and ready data
url = "https://www.usaspending.gov/state"


In [26]:
def usaspending_scraper(url=None):

    # Download the webpage
    page = requests.get(url)

    # If a connection was reached
    if page.status_code == 200:

        # Parse
        soup = BeautifulSoup(page.content, 'html.parser')

        # Pull Table
        table = soup.select("#main-content > div > div:nth-child(2) > div.state-landing__results > table")[0].get_text()

        # Return data
        return [table]


In [27]:
page = requests.get(url)

In [28]:
soup = BeautifulSoup(page.content, 'html.parser')

In [29]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="/"/>
<!-- Facebook Open Graph -->
<meta content="website" property="og:type">
<!-- Twitter Card Tags -->
<meta content="summary_large_image" name="twitter:card">
<meta name="twitter:domain" value="usaspending.gov"/>
<meta name="twitter:site" value="@usaspending"/>
<title>USAspending.gov</title>
<link href="https://api.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.css" rel="stylesheet"/>
<link href="img/favicon.ico" rel="shortcut icon"/>
<script async="" id="_fed_an_ua_tag" src="https://dap.digitalgov.gov/Universal-Federated-Analytics-Min.js?agency=TREAS&amp;subagency=FS" type="text/javascript"></script>
<script src="https://api.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>
<!-- Google Analytics -->
<script>
        window.ga = window.ga || function () { (ga.q = ga.q || []).push(arguments) }; ga.l = +new Date;
        ga('create', 'UA-6